### Import Libraries

In [1]:
from pyspark.sql.functions import col, input_file_name, to_date, to_timestamp, regexp_extract
from pyspark.sql.types import MapType, StringType, StructField, StructType

from lib.spark import get_spark_session

### Configure Session

In [2]:
spark = get_spark_session(driver="spark://localhost:7077")
sc = spark.sparkContext

22/05/19 13:57:21 WARN Utils: Your hostname, Jordans-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 10.0.0.140 instead (on interface en0)
22/05/19 13:57:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/jordan.yaker/.ivy2/cache
The jars for the packages stored in: /Users/jordan.yaker/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.spark#spark-streaming-kinesis-asl_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a638dc95-a749-4de3-aea6-74c85adac974;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
	found io.delta#delta-core_2.12;1.2.1 in central
	found io.delta#delta-storage;1.2.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found c

### Configure Job

In [35]:
INPUT_PATH = "s3a://oc-dev-internal-data-lake-kinesis-cdc/"
INPUT_PATH_REGEX = r"s3a:\/\/oc-dev-internal-data-lake-kinesis-cdc\/([a-zA-Z0-9_]+?)\/([a-zA-Z0-9\-]+?)\/.*"
INPUT_SCHEMA = StructType([
    StructField("data", MapType(StringType(), StringType()), False),
    StructField("metadata", MapType(StringType(), StringType()), False)
])

OUTPUT_PATH = "s3a://oc-dev-internal-data-lake-bronze/changes/"

### Etract Data

In [14]:
input = spark.read.json(INPUT_PATH, INPUT_SCHEMA, recursiveFileLookup=True)

### Transform Data

In [38]:
df = input.withColumn("filename", input_file_name()) \
          .select(regexp_extract("filename", INPUT_PATH_REGEX, 2).cast("date").alias("day_id"),
                  regexp_extract("filename", INPUT_PATH_REGEX, 1).alias("table_name"),
                  to_timestamp(col("metadata")['timestamp'], "yyyy-MM-dd'T'HH:mm:ss.SSSSSSVV").alias("timestamp"),
                  "data")

### Load Data

In [40]:
df.write \
  .format("delta") \
  .mode("append") \
  .save(OUTPUT_PATH)